
<center>

# dask-image: distributed image processing for large data

## Presenter: Genevieve Buckley
<img src="imgs/dask-icon.svg#thumbnail" alt="dask logo" width="100"/>
</center>

# Who needs dask-image?

If you're using `numpy` and/or `scipy.ndimage` and are running out of RAM, dask-image is for you.

## Two main use cases
1. Batch processing
2. Large field of view

# Motivating examples

* Sentinel satellite data
* Individual neurons within the brain

<center>
<img src="imgs/motivating-examples.png#thumbnail" alt="Satellite data and brain neurons" width="1000"/></center>


# Getting started
https://github.com/dask/dask-image/

## conda
```
conda install -c conda-forge dask-image
```

## pip

```
pip install dask-image
```


# What's included?

* imread
* ndfilters
* ndfourier
* ndmeasure
* ndmorph



# Function coverage

<img src="imgs/function-coverage-table.png#thumbnail" alt="Table of function coverage: scipy.ndimage compared to dask-image http://image.dask.org/en/latest/coverage.html" width="900"/>


# GPU support

Latest release includes GPU support for the modules:
* ndfilters
* imread

Still to do: ndfourier, ndmeasure, ndmorph*

*Done, pending cupy PR #3907

# GPU benchmarking

| Architecture    | Time      |
|-----------------|-----------|
| Single CPU Core | 2hr 39min |
| Forty CPU Cores | 11min 30s |
| One GPU         | 1min 37s  |
| Eight GPUs      | 19s       |

http://matthewrocklin.com/blog/work/2019/01/03/dask-array-gpus-first-steps


# Let's build a pipeline!

1. Reading in data
2. Filtering images
3. Segmenting objects
4. Morphological operations
5. Measuring objects

We used image set [BBBC039v1](https://bbbc.broadinstitute.org/bbbc/BBBC039) Caicedo et al. 2018, available from the Broad Bioimage Benchmark Collection [Ljosa et al., Nature Methods, 2012](http://dx.doi.org/10.1038/nmeth.2083). 

https://bbbc.broadinstitute.org/BBBC039

<center>
<img src="imgs/BBBC039-example-image.png#thumbnail" alt="BBBC039 image of flourescent nuclei" width="700"/>


# 1. Reading in data


In [20]:
from dask_image.imread import imread

images = imread('data/BBBC039/images/*.tif')
# images_on_gpu = imread('data/BBBC039/images/*.tif', arraytype="cupy")

images

,Array,Chunk
Bytes,144.77 MB,723.84 kB
Shape,"(200, 520, 696)","(1, 520, 696)"
Count,600 Tasks,200 Chunks
Type,uint16,numpy.ndarray


# 2. Filtering images
Denoising images with a small blur can improve segmentation later on.

In [21]:
from dask_image import ndfilters

smoothed = ndfilters.gaussian_filter(images, sigma=[0, 1, 1])

# 3. Segmenting objects
Pixels below the threshold value are background.

In [22]:
absolute_threshold = smoothed > 150

In [23]:
# Let's have a look at the images
%gui qt

In [24]:
import napari
viewer = napari.Viewer() 
viewer.add_image(absolute_threshold)
viewer.add_image(images)  

<Image layer 'images' at 0x7fa1495aa450>

# 3. Segmenting objects (continued)

Pixels below the threshold value are background.


In [25]:
thresh = ndfilters.threshold_local(smoothed, images.chunksize)
threshold_images = smoothed > thresh

In [26]:
viewer.add_image(threshold_images)

<Image layer 'threshold_images' at 0x7fa1421e6790>

# 4. Morphological operations

These are operations on the sahpe of a binary image.

https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html



## Erosion
<center>
<img src="imgs/erosion.png#thumbnail" alt="Erosion, binary morphological operation 
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
" width="500"/>

## Dilation

<center>
<img src="imgs/dilation.png#thumbnail" alt="Dilation, binary morphological operation 
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
" width="500"/>


A morphological close operation is an erosion, followed by a dilation.

<center>
<img src="imgs/opening.png#thumbnail" alt="Opening, binary morphological operation 
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
" width="500"/>


In [27]:
from dask_image import ndmorph
import numpy as np

structuring_element = np.array([
    [[0, 0, 0], [0, 0, 0], [0, 0, 0]],
    [[0, 1, 0], [1, 1, 1], [0, 1, 0]],
    [[0, 0, 0], [0, 0, 0], [0, 0, 0]]])
binary_images = ndmorph.binary_opening(threshold_images structure=structuring_element) 



# 5. Measuring objects
Each image has many individual nuclei, so for the sake of time we'll measure a small subset of the data.

In [33]:
from dask_image import ndmeasure

# Create labelled mask
label_images, num_features = ndmeasure.label(binary_images[:3], structuring_element)
index = np.arange(num_features)
print("Number of nuclei:", num_features.compute())

Number of nuclei: 268


In [30]:
# Let's look at the labels
viewer.add_labels(label_images)
viewer.dims.set_point(0, 0)

In [32]:
# Measure objects in images
area = ndmeasure.area(images[:3], label_images, index)
mean_intensity = ndmeasure.mean(images[:3], label_images, index)
stdev_intensity = ndmeasure.standard_deviation(images[:3], label_images, index)

# The full pipeline

```python
import numpy as np
from dask_image.imread import imread
from dask_image import ndfilters, ndmorph, ndmeasure

images = imread('data/BBBC039/images/*.tif')
smoothed = ndfilters.gaussian_filter(images, sigma=[0, 1, 1])
thresh = ndfilters.threshold_local(smoothed, blocksize=images.chunksize)
threshold_images = smoothed > thresh
structuring_element = np.array([[[0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 1, 0], [1, 1, 1], [0, 1, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0]]])
binary_images = ndmorph.binary_closing(threshold_image)
label_images, num_features = ndmeasure.label(binary_image)
index = np.arange(num_features)
area = ndmeasure.area(images, label_images, index)
mean = ndmeasure.mean(images, label_images, index)
stdev = ndmeasure.standard_deviation(images, label_images, index)
```

# Custom functions

What if you want to do something that isn't included?

* scikit-image [apply_parallel()](https://scikit-image.org/docs/dev/api/skimage.util.html#skimage.util.apply_parallel)
* dask [map_overlap](https://docs.dask.org/en/latest/array-overlap.html?highlight=map_overlap#dask.array.map_overlap) / [map_blocks](https://docs.dask.org/en/latest/array-api.html?highlight=map_blocks#dask.array.map_blocks)
* dask [delayed](https://docs.dask.org/en/latest/delayed.html) 

# Scaling up computation

* [dask-distributed](https://distributed.dask.org/en/latest/)

```python
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client.cluster

```

# dask-image

* Install: `conda` or `pip install dask-image`
* Documentation: https://dask-image.readthedocs.io
* GitHub: https://github.com/dask/dask-image/

 <center>
 <img src="imgs/dask-icon.svg#thumbnail" alt="dask logo" width="300"/></center>
 

```python
# CPU example
import numpy as np
import dask.array as da
from dask_image.ndfilters import convolve

s = (10, 10)
a = da.from_array(np.arange(int(np.prod(s))).reshape(s), chunks=5)
w = np.ones(a.ndim * (3,), dtype=np.float32)
result = convolve(a, w)
result.compute()
```

```python
# Same example moved to the GPU
import cupy  # <- import cupy instead of numpy
import dask.array as da
from dask_image.ndfilters import convolve

s = (10, 10)
a = da.from_array(cupy.arange(int(cupy.prod(cupy.array(s)))).reshape(s), chunks=5)  # <- cupy dask array
w = cupy.ones(a.ndim * (3,))  # <- cupy dask array
result = convolve(a, w)
result.compute()
```